In [1]:
from typing import Dict, List, Optional, Callable
import json
from autogen_utils import peep, fprint
from autogen_utils import (
    peep,
    get_llm_config,
    get_func_args,
)

# @tool("search", return_direct=True)
# def search_apis(query: str) -> str:
#     """Searches the API for the query."""
#     return "Results"

In [2]:
from typing import Optional, Type
import os


def get_file():
    # Get the current working directory
    file_path = os.path.join(os.getcwd(), "20160728.txt")

    with open(file_path, "r") as f:
        content = f.read()

    return content


content = get_file()

In [3]:
# 예시: 만약 아래와 같은 function이 있다면 llm_config는 다음과 같이 만들어진다.
# argument인 query에 대해 description이 필요할 때는 {"query": |DESCRIPTION|}와 같은 dict를 사용한다.


def search_api(query: str) -> str:
    """Searches the API for the query."""
    return f"Results for query {query}"


descriptions = {"query": "Query for search"}

get_llm_config(model="gpt-4-1106-preview", functions=[search_api], descriptions=descriptions)

config_list_mixtral = [{"model": "mistral", "base_url": "http://0.0.0.0:8000"}]

## Defining functions to be used by Agents.


In [4]:
# dsm5 criteria를 json으로 부터 불러들인다.
dsm5_criteria = json.load(open("dsm_5.json"))

In [8]:
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain_core.vectorstores import VectorStoreRetriever


embedding = OpenAIEmbeddings()

docs = [Document(page_content=k + "\n" + v["criteria"], metadata={"diagnosis": k}) for k, v in dsm5_criteria.items()]

# db = Chroma.from_documents(docs, persist_directory="./db", embedding=embedding)

db = Chroma.from_documents(docs, persist_directory="./db", embedding=embedding)

In [11]:
match = "depressive disorder"
dsm5 = dsm5_criteria


descriptions = {
    "match_diagnosis": "Diagnosis for which similar diagnoses are to be identified.",
    "diagnosis": "Diagnosis you want to check whether is official DSM-5 term",
    "diagnoses": "the list of possible diagnoses proposed by presenter and the critic",
}


def find_similar_diagnoses(match_diagnosis: str) -> List[str]:
    """Find diagnoses which are conceptually similar to or belong to the same category as the given diagnosis. Diagnoses are retrieved from the vector store or by matching the first two digits of the ICD-10 code."""

    match_diagnosis = match_diagnosis.lower()
    retriever = db.as_retriever(search_kwargs={"k": 5})

    if match_diagnosis in dsm5:
        match_code = dsm5[match_diagnosis]["code"][:2] if dsm5[match_diagnosis]["code"] else None
    else:
        match_code = None

    matched = [doc.metadata["diagnosis"] for doc in retriever.invoke(match_diagnosis)]
    if match_code:
        for diagnosis, v in dsm5.items():
            code = v["code"]
            if (code is not None) and (code.startswith(match_code)):
                matched.append(diagnosis)
    return matched


def find_synonym(match_diagnosis: str) -> str:
    """Find the mose close diagnosis which are conceptually similar to the given diagnosis.
    Useful for when the user do not use the official DSM-5 diagnosis"""

    match_diagnosis = match_diagnosis.lower()
    retriever = db.as_retriever(search_kwargs={"k": 3})

    matched = [doc.metadata["diagnosis"] for doc in retriever.invoke(match_diagnosis)]

    return matched[0] if matched else None

In [13]:
match_diagnosis = "PTSD"
find_synonym(match_diagnosis)

'posttraumatic stress disorder'

In [14]:
def check_standard_diagnosis(diagnosis: str) -> bool:
    """Check if diagnosis is a valid DSM-5 diagnosis."""
    return diagnosis.lower().split(",")[0] in dsm5.keys()


def get_dsm5_criteria(diagnosis: str, stop_words: List[str] = ["specify", "specifier"]) -> str:
    """Fetch DSM-5 diagnostic criteria for a given diagnosis."""

    if not check_standard_diagnosis(diagnosis):
        diagnosis = find_synonym(diagnosis)
    temp = dsm5[diagnosis.lower()]["criteria"]

    try:
        location = min(temp.lower().index(substr) for substr in stop_words)
    except Exception:
        location = -1
    return f"{diagnosis.upper()}:\n{temp[:location]}\n"


def get_all_criteria(diagnoses: str) -> str:
    temp = [get_dsm5_criteria(diagnosis.strip()) for diagnosis in diagnoses.split(",")]
    return "\n".join(temp)

In [20]:
searcher_llm_config = get_llm_config(
    functions=[find_similar_diagnoses], descriptions=descriptions, model="gpt-4-1106-preview"
)
verifier_llm_config = get_llm_config(
    functions=[get_all_criteria], descriptions=descriptions, model="gpt-4-1106-preview"
)
llm_config = get_llm_config(functions=[check_standard_diagnosis], model="gpt-4-1106-preview")

llm_config["config_list"] = config_list_mixtral
verifier_llm_config["config_list"] = config_list_mixtral

bare_llm_config = {"config_list": config_list_mixtral, "timeout": 120}

## Build agents


In [21]:
import autogen


def build_user_proxy(
    message: str, functions: List[Callable] = [], human_input_mode: str = "NEVER", max_consecutive_auto_reply: int = 10
) -> autogen.UserProxyAgent:
    agent = autogen.UserProxyAgent(
        name="Admin",
        system_message=message,
        code_execution_config=False,
        is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
        human_input_mode=human_input_mode,
        max_consecutive_auto_reply=max_consecutive_auto_reply,
    )

    if len(functions) > 0:
        function_map = {}
        for func in functions:
            function_map[func.__name__] = func

        agent.register_function(function_map=function_map)

    return agent


def build_assistent(name: str, llm_config: Dict, message: str):
    return autogen.AssistantAgent(name=name, llm_config=llm_config, system_message=message)

In [22]:
admin = build_user_proxy(
    "You are a psychiatry professor.",
    functions=[find_similar_diagnoses, get_all_criteria, check_standard_diagnosis],
)

presenter = build_assistent(
    name="presenter",
    llm_config=llm_config,
    message="You are the primary presenter. You will submit first diagnostic impression of the case. Your opinion will be used as the basis for further discussion. You should first provide a list of 5 or 6 possible diagnoses. Only after receiving the reply from the 'verifier', you may select the most probable diagnosis among the probable diagnoses that you and the 'critic' had proposed. In this process, you have to provide why you selected this diagnosis and why you ruled out other diagnoses. Try to use standard DSM-5 diagnosis names. You may verify this using the supplied function.",
)

critic = build_assistent(
    name="critic",
    llm_config=llm_config,
    message="""You will criticise the diagnoses proposed by the presenter and provide your own list of another possible diagnoses. You don't like the diagnostic procedure of mechanically applying DSM criteria, and believe that diagnosis should be made by inferring deep psychology based on the patients' past experiences, surrounding circumstances, and their personality. But, your criticism should also be based on the solid rationale. After receiving feedback from the 'verifier' and the 'presenter', you may revise your opinion about the diagnosis. Try to use standard DSM-5 diagnosis names. You may verify this using the supplied function. After satisfactory discussion with other agents, if it is determined that you reached an agreement with the presenter, send a signal 'TERMINATE'""",
)

# searcher = build_assistent(
#     name="searcher",
#     llm_config=searcher_llm_config,
#     message="You should search for other diagnoses similar to the diagnosis made by the presenter and critic using the supplied function and deliver the list of possible diagnoses to the verifier.",
# )

verifier = build_assistent(
    name="verifier",
    llm_config=verifier_llm_config,
    message="""You will receive the list of possible diagnoses from both the 'presenter' and the 'critic'. You first have to merge these lists into one list without duplication . Your task is to verify whether the case material comply with the DSM-5 criteria for each diagnosis contained in this list. You will return the result in a table with three columns: 'diagnosis', 'whether the case met the DSM-5 criteria (in True or False)', 'reason for not meeting the criteria (only when false)'""",
)

In [24]:
groupchat = autogen.GroupChat(
    agents=[admin, presenter, critic, verifier],
    messages=[],
    max_round=20,
    speaker_selection_method="round_robin",
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=bare_llm_config)

In [26]:
admin.initiate_chat(
    manager,
    message=f"Read the following case report. \n\n {get_file()} \n\n Thoroughly discuss the probable diagnoses compatible with the case material.",
    llm_config=llm_config,
)

Admin (to chat_manager):

Read the following case report. 

 상기환자는 병전 발랄하고 밝은 성격이었음. 현재 대학교 휴학 중이며 어머니, 오빠, 남동생과 같이 거주중임. 환자가 초등학교 5학년 시절 부모가 이혼을 하였고 환자는 아버지를 따라 태국으로 가서 4년간 같이 살게 되었음. 하지만 아버지의 지속된 폭력적인 모습으로 남매 모두가 아버지와 살 수 없다며 한국으로 귀국하여 어머니와 함께 지내왔음. 
 환자는 2014년도까지 한국에서 어머니와 함께 일상생활을 잘 지내왔고 아버지가 한국으로 와서 환자와 간헐적으로 연락을 해 만난 적도 있음. 2014년 12월 말경 아버지의 선교사 일을 돕기 위해 아버지와 같이 태국으로 가게 되었고 2주 동안은 큰 문제없이 생활하였고 한국에 있는 어머니와 연락을 할 때도 별 문제 없었음. 2015년 1월 환자는 5년간 사귄 태국에 살고 있는 중국인 남자친구를 만나기 위해 푸켓으로 가게 되었음. 환자의 남자친구는 푸켓에서 여행 가이드를 하고 있었는데 여행 패키지 빈자리에 환자가 합류하여 같이 여행을 다녔음. 여행 일정 중 스트립 쇼를 하는 바에 방문하는 것이 있었고 그곳에서 퇴폐적인 문화를 보면서 환자는 충격을 크게 받았음. 환자는 스트립 바에서 주는 칵테일을 1-2잔 마신 이후 정신을 잃었다고 하며 일어나서 보니 옷이 벗겨진 채 남자친구와 호텔방에 있었음. 당시 환자는 처음 성관계를 하게 되었다고 하며 어느 정도 예상은 하고 여행을 간 것이긴 하지만 충격은 있었음. 이후 환자는 본인 주량에 비해 적은 양의 술을 마셨는데 정신이 없어진 것 같다고 하며 술에 무언가를 타지 않았을까 생각했었음. 환자는 이 일이 있은 이후부터 생각이 혼란스럽고 누가 환자의 이름을 부르는 소리가 들리는 auditory hallucination 증상 발생하였음. 또한 환자는 어머니에게 메신저로 단어로만 이야기를 하며 친구들이 환자를 놀리는 것 같은 paranoid ideation 보이며 말이 점차 없어졌음. 

APIConnectionError: Connection error.

In [ ]:
for name, content in [(m["name"], m["content"]) for m in manager.chat_messages[presenter][1:] if m["content"] != ""]:
    if content not in ["True", "False", None]:
        print(name.title(), ":")
        print(content)
        print()

In [62]:
manager.chat_messages[presenter]

[{'content': 'Read the following case report. \n\n 상기환자는 병전 발랄하고 밝은 성격이었음. 현재 대학교 휴학 중이며 어머니, 오빠, 남동생과 같이 거주중임. 환자가 초등학교 5학년 시절 부모가 이혼을 하였고 환자는 아버지를 따라 태국으로 가서 4년간 같이 살게 되었음. 하지만 아버지의 지속된 폭력적인 모습으로 남매 모두가 아버지와 살 수 없다며 한국으로 귀국하여 어머니와 함께 지내왔음. \n 환자는 2014년도까지 한국에서 어머니와 함께 일상생활을 잘 지내왔고 아버지가 한국으로 와서 환자와 간헐적으로 연락을 해 만난 적도 있음. 2014년 12월 말경 아버지의 선교사 일을 돕기 위해 아버지와 같이 태국으로 가게 되었고 2주 동안은 큰 문제없이 생활하였고 한국에 있는 어머니와 연락을 할 때도 별 문제 없었음. 2015년 1월 환자는 5년간 사귄 태국에 살고 있는 중국인 남자친구를 만나기 위해 푸켓으로 가게 되었음. 환자의 남자친구는 푸켓에서 여행 가이드를 하고 있었는데 여행 패키지 빈자리에 환자가 합류하여 같이 여행을 다녔음. 여행 일정 중 스트립 쇼를 하는 바에 방문하는 것이 있었고 그곳에서 퇴폐적인 문화를 보면서 환자는 충격을 크게 받았음. 환자는 스트립 바에서 주는 칵테일을 1-2잔 마신 이후 정신을 잃었다고 하며 일어나서 보니 옷이 벗겨진 채 남자친구와 호텔방에 있었음. 당시 환자는 처음 성관계를 하게 되었다고 하며 어느 정도 예상은 하고 여행을 간 것이긴 하지만 충격은 있었음. 이후 환자는 본인 주량에 비해 적은 양의 술을 마셨는데 정신이 없어진 것 같다고 하며 술에 무언가를 타지 않았을까 생각했었음. 환자는 이 일이 있은 이후부터 생각이 혼란스럽고 누가 환자의 이름을 부르는 소리가 들리는 auditory hallucination 증상 발생하였음. 또한 환자는 어머니에게 메신저로 단어로만 이야기를 하며 친구들이 환자를 놀리는 것 같은 paranoid ideation 보이며 말이 점차 없어졌음. 이를 이상하게 여

In [22]:
get_dsm5_criteria("schizophrenia")

{'code': 'F20.9',
 'criteria': 'A. Two (or more) of the following, each present for a significant portion of time during a 1-month period (or less if successfully treated). At least one of these must be (1), (2), or (3):\n1. Delusions. 2. Hallucinations. 3. Disorganized speech (e.g., frequent derailment or incoherence).\n4. Grossly disorganized or catatonic behavior. 5. Negative symptoms (i.e., diminished emotional expression or avolition).\nB. For a significant portion of the time since the onset of the disturbance, level of functioning in one or more major areas, such as work, interpersonal relations, or self-care, is markedly below the level achieved prior to the onset (or when the onset is in childhood or adolescence, there is failure to achieve expected level of interpersonal, academic, or occupational functioning).\nC. Continuous signs of the disturbance persist for at least 6 months. This 6-month period must include at least 1 month of symptoms (or less if successfully treated) 